<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Импорт-используемых-библиотек" data-toc-modified-id="Импорт-используемых-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт используемых библиотек</a></span></li><li><span><a href="#Чтение-файла" data-toc-modified-id="Чтение-файла-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Чтение файла</a></span></li><li><span><a href="#Информация-о-файле" data-toc-modified-id="Информация-о-файле-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Информация о файле</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Удаление-дубликатов" data-toc-modified-id="Удаление-дубликатов-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Удаление дубликатов</a></span></li><li><span><a href="#Изменение-наименований-столбцов" data-toc-modified-id="Изменение-наименований-столбцов-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>Изменение наименований столбцов</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

___

# Описание данных
Признаки: 
- Пол
- Возраст
- Зарплата застрахованного
- Количество членов его семьи.

Целевой признак:
- Страховые выплаты

## Загрузка данных

### Импорт используемых библиотек

In [1]:
import pandas as pd
import ydata_profiling
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

### Чтение файла

In [2]:
try:
    df = pd.read_csv('insurance.csv', sep=',')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv', sep=',')

### Информация о файле

Воспользуемся библиотекой Pandas Profiling

In [3]:
ydata_profiling.ProfileReport(df, explorative=True).to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Выводы по данным:
- Пропуски отсутствуют
- Дубликатов - 147
- Представительство по полу у клиентов близкое к равному (разница в 0.2%)
- Медианный возраст клиентов - 30 лет
- По уровню зарплат наблюдается равномерное распределение (медиана - 40200)
- Треть от всех клиентов (30.3%) не указали членов семьи
- Страховые выплаты полагались только 11.3% от всех клиентов
- Наблюдается неявная корреляция между страховыми выплатами и возрастом

### Предобработка данных

#### Удаление дубликатов

In [4]:
df = df.drop_duplicates()

In [5]:
df.shape

(4847, 5)

Дубликаты успешно удалены

#### Изменение наименований столбцов

Изменим на латиницу имена столбцов для удобства (и по правилам хорошего тона)

In [6]:
df.columns = ['gender', 'age', 'salary', 'family_members', 'insurance']

In [7]:
df.columns

Index(['gender', 'age', 'salary', 'family_members', 'insurance'], dtype='object')

### Вывод

После EDA были удалены дубликаты и изменены названия столбцов.

## Умножение матриц

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

При решении воспользуемся следующими основными свойствами матриц:

1. Транспонирование произведения матриц равно произведению транспонированных матриц в обратном порядке. 
$$
(AB)^T = B^TA^T
$$


2. Обратная матрица от произведения двух матриц равна произведению обратных матриц в обратном порядке.
$$
(AB)^{-1} = (B^{-1}A^{-1})
$$


3. Умножение матрицы на её обратную матрицу дает единичную матрицу.
$$
AA^{-1} = A^{-1}A = E
$$


4. Умножение любой матрицы на единичную матрицу дает ту же самую матрицу.
$$
EA = AE = A
$$

Умножим матрицу признаков X на обратимую матрицу P:

$$
a' = XPw'
$$


$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$

Используя основные свойства матрицы, описанные выше, проведем вычисления:

$$
w' = ((XP)^T XP)^{-1} (XP)^T y = \\
= (P^T X^T X P)^{-1} (XP)^T y = \\
= ((P^T X^T X) P)^{-1} (XP)^T y = \\
= P^{-1} (P^T X^T X)^{-1} P^T X^T y = \\
= P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y = \\
= P^{-1} (X^T X)^{-1} E X^T y = \\
= P^{-1} (X^T X)^{-1} X^T y = \\
= P^{-1} w
$$


Подставим результат в формулу предсказания и получим:

$$
a' = X P w' = X P P^{-1} w = X E w = Xw = a
$$


Качество модели не изменится.   

## Алгоритм преобразования

Преобразование данных можно провести используя умножение признаков на обратимую матрицу и уже на этих данных обучить модель. Матрицу сделать случайной, при этом добавить шаг проверки матрицы на обратимостью.

- Выделим целевой признак и признаки
- Генерируем случайную матрицу, размер которой соответствует количеству признаков
- Генерируем обратную матрицу

**Обоснование**

Математическое обоснование алгоритма приведено в предыдущем пункте.

## Проверка алгоритма

In [8]:
# Матрица признаков X
X = df.drop('insurance', axis=1).values

# Вектор целевого признака y
y = df['insurance'].values

#Размер матрицы
matrix_size = X.shape[1]

In [9]:
# Матрица преобразования matrix
matrix = np.random.normal(0, 0.1, size=(matrix_size, matrix_size))

matrix

array([[-0.17034998,  0.04464843, -0.03616661,  0.01022589],
       [ 0.11224676,  0.1098913 , -0.04847549, -0.08070598],
       [ 0.12263216, -0.08470941, -0.02573198,  0.08490483],
       [ 0.10648413,  0.07012757,  0.03622193, -0.0466934 ]])

In [10]:
#Проверим сгенерированную матрицу на обратимость
inverse_matrix = np.linalg.inv(matrix) 

inverse_matrix

array([[-1.6658386 ,  1.4008874 ,  2.70558111,  2.13353945],
       [12.49096817, -1.17305981,  5.54079309, 14.83816375],
       [-0.19599238, -9.78832697, -2.31345596, 12.66878689],
       [14.8088715 , -6.16026315, 12.69699591, 15.56194835]])

In [11]:
# Применяем преобразование данных
X_transformed = np.dot(X, matrix)

In [12]:
# Обучение линейной регрессии на исходных данных
reg_before = LinearRegression()
reg_before.fit(X, y)

# Обучение линейной регрессии на преобразованных данных
reg_after = LinearRegression()
reg_after.fit(X_transformed, y)

# Предсказания на исходных данных и преобразованных данных
y_pred_before = reg_before.predict(X)
y_pred_after = reg_after.predict(X_transformed)

# Вычисляем метрику R2 для оценки качества на исходных данных и преобразованных данных
r2_before = r2_score(y, y_pred_before)
r2_after = r2_score(y, y_pred_after)

print(f"R2 score до преобразования: {r2_before}")
print(f"R2 score после преобразования: {r2_after}")

R2 score до преобразования: 0.4302010044852067
R2 score после преобразования: 0.4302010044852126


## Вывод:

Результат метрики одинаковый для исходных и преоразованных данных. Соответвенно, защита данных клиентов никак не повлияла на результаты работы модели.